In [1]:
## base functions for xgboost, catboost and lightgbm are to be added
## has tuning using bayesian framework
## also will follow up with LIME/tree interpreters for the same

## sample data used is FLight delays dataset ##

# lightgbm done
# xgboost needs to be setup
# start on catboost

In [2]:
## importing the various packages

# clear the workspace
%reset -f

# print list of files in directory
import os
print(os.listdir())

# print/display all plots inline
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# the base packages
import collections # for the Counter function
import csv # for reading/writing csv files
import pandas as pd, numpy as np, time, gc
import ast # for the AST segment, parsing json to table and so on

# the various packages/modules used across processing (sklearn), modelling (lightgbm) and bayesian optimization (hyperopt, bayes_opt)
from sklearn.model_selection import train_test_split
from sklearn import metrics, preprocessing
from sklearn.cross_validation import cross_val_score, StratifiedKFold, StratifiedShuffleSplit
from bayes_opt import BayesianOptimization
from tqdm import tqdm
import lightgbm as lgb
from hyperopt import hp, tpe, STATUS_OK, fmin, Trials
from hyperopt.fmin import fmin
from hyperopt.pyll.stochastic import sample

# Evaluation of the model
from sklearn import model_selection
from sklearn.model_selection import KFold

# define the global variables used later
MAX_EVALS = 10 # number of iterations/parameter sets created towards tuning
N_FOLDS = 5 # number of cv folds
random_seed = 1 # the value for the random state used at various points in the pipeline

['.ipynb_checkpoints', 'ABI_modelling_snippets.ipynb', 'airlines.csv', 'airports.csv', 'flights.csv', 'flights_sample.csv', 'gbm_trials.csv', 'hyperparameter-optimization-master']


C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
## function to get frequency count of elements in a vector/list
def freq_count (input_vector):
    return collections.Counter(input_vector)

## function to create 
def prepare_data(input_file_path = 'flights_sample.csv', response = 'ARRIVAL_DELAY'):
    train = pd.read_csv(input_file_path)
    train = train.sample(frac = 0.1, random_state = random_seed)
    train = train[["MONTH","DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT","AIR_TIME", "DEPARTURE_TIME","DISTANCE","ARRIVAL_DELAY"]]
    train.dropna(inplace = True)
    print(train.shape)
    
    categorical_columns = train.select_dtypes(include=['object']).columns
    for column in tqdm(categorical_columns):
        le = preprocessing.LabelEncoder()
        train[column] = le.fit_transform(train[column].astype(str))
    
    train[response] = (train[response] > 25)*1
    print(freq_count(train[response]))

    y = train[response].values
    X = train.drop([response], axis = 1)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = random_seed)

    return X_train, X_test, y_train, y_test

In [4]:
## preparing the different train/test features/labels datasets
X_train, X_test, y_train, y_test = prepare_data()

(5707, 11)


100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 175.72it/s]


Counter({0: 4997, 1: 710})


In [5]:
# Model with default hyperparameters
model = lgb.LGBMClassifier()
model

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
        learning_rate=0.1, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=100,
        n_jobs=-1, num_leaves=31, objective=None, random_state=None,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=0)

In [6]:
# baseline model with default hyperparameters

from sklearn.metrics import roc_auc_score
from timeit import default_timer as timer

start = timer()
model.fit(X_train, y_train)
train_time = timer() - start

predictions = model.predict_proba(X_test)[:, 1]
auc = roc_auc_score(y_test, predictions)

print('The baseline score on the test set is {:.4f}.'.format(auc))
print('The baseline training time is {:.4f} seconds'.format(train_time))

# Create a lgb dataset
train_set = lgb.Dataset(X_train, label = y_train)

The baseline score on the test set is 0.6792.
The baseline training time is 0.1197 seconds


In [7]:
## RANDOM SEARCH TUNING FOR LIGHTGBM ##

## start of random tuning module ##

import random

# Hyperparameter grid
param_grid = {
    'class_weight': [None, 'balanced'],
    'boosting_type': ['gbdt', 'goss', 'dart'],
    'num_leaves': list(range(50, 300)),
    'learning_rate': list(np.logspace(np.log(0.005), np.log(0.2), base = np.exp(1), num = 1000)),
    'subsample_for_bin': list(range(20000, 300000, 20000)),
    'min_child_samples': list(range(20, 500, 5)),
    'reg_alpha': list(np.linspace(0, 1)),
    'reg_lambda': list(np.linspace(0, 1)),
    'colsample_bytree': list(np.linspace(0.6, 1, 10))
}

# Subsampling (only applicable with 'goss')
subsample_dist = list(np.linspace(0.5, 1, 100))

# # learning rate distribution
# # graph plotted for the same
# plt.hist(param_grid['learning_rate'], color = 'r', edgecolor = 'k');
# plt.xlabel('Learning Rate', size = 14); plt.ylabel('Count', size = 14); plt.title('Learning Rate Distribution', size = 18);

# # number of leaves distribution
# # graph plotted for the same
# plt.hist(param_grid['num_leaves'], color = 'm', edgecolor = 'k')
# plt.xlabel('Learning Number of Leaves', size = 14); plt.ylabel('Count', size = 14); plt.title('Number of Leaves Distribution', size = 18);

# ## sample prediction with random params
#####################################################################################################
# # Randomly sample parameters for gbm
# # sample snippet to return a sample set of parameters
# params = {key: random.sample(value, 1)[0] for key, value in param_grid.items()}
# params['subsample'] = random.sample(subsample_dist, 1)[0] if params['boosting_type'] != 'goss' else 1.0
# print(params)

# # Perform cross validation with N folds
# r = lgb.cv(params, train_set, num_boost_round = 10000, nfold = N_FOLDS, metrics = 'auc', 
#            early_stopping_rounds = 100, verbose_eval = False, seed = random_seed)

# # Highest score
# r_best = np.max(r['auc-mean'])

# # Standard deviation of best score
# r_best_std = r['auc-stdv'][np.argmax(r['auc-mean'])]

# print('The maximium ROC AUC on the validation set was {:.5f} with std of {:.5f}.'.format(r_best, r_best_std))
# print('The ideal number of iterations was {}.'.format(np.argmax(r['auc-mean']) + 1))
######################################################################################################

# Dataframe to hold cv results
random_results = pd.DataFrame(columns = ['loss', 'params', 'iteration', 'estimators', 'time'],
                       index = list(range(MAX_EVALS)))

# the objective function that returns a minimization value computed by taking the difference of 1 and roc-auc
def random_objective(params, iteration, n_folds = N_FOLDS):
    """Random search objective function. Takes in hyperparameters
       and returns a list of results to be saved."""

    start = timer()
    
    # Perform n_folds cross validation
    cv_results = lgb.cv(params, train_set, num_boost_round = 10000, nfold = n_folds, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = random_seed)
    end = timer()
    best_score = np.max(cv_results['auc-mean'])
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Boosting rounds that returned the highest cv score
    n_estimators = int(np.argmax(cv_results['auc-mean']) + 1)
    
    # Return list of results
    return [loss, params, iteration, n_estimators, end - start]

In [8]:
%%capture

random.seed(random_seed)

# Iterate through the specified number of evaluations
for i in range(MAX_EVALS):
    
    # Randomly sample parameters for gbm
    params = {key: random.sample(value, 1)[0] for key, value in param_grid.items()}
    print(params)
    
    if params['boosting_type'] == 'goss':
        # Cannot subsample with goss
        params['subsample'] = 1.0
    else:
        # Subsample supported for gdbt and dart
        params['subsample'] = random.sample(subsample_dist, 1)[0]
        
    results_list = random_objective(params, i)
    # Add results to next row in dataframe
    random_results.loc[i, :] = results_list

In [9]:
# Sort results by best validation score
random_results.sort_values('loss', ascending = True, inplace = True)
random_results.reset_index(inplace = True, drop = True)

# print the best set of hyper-parameters generated
print(random_results.loc[0, 'params'])

# Find the best parameters and number of estimators
best_random_params = random_results.loc[0, 'params'].copy()
best_random_estimators = int(random_results.loc[0, 'estimators'])
best_random_model = lgb.LGBMClassifier(n_estimators=best_random_estimators, n_jobs = -1, 
                                       objective = 'binary', **best_random_params, random_state = random_seed)

# Fit on the training data
best_random_model.fit(X_train, y_train)

# Make test predictions
predictions = best_random_model.predict_proba(X_test)[:, 1]

print('The best model from random search scores {:.4f} on the test data.'.format(roc_auc_score(y_test, predictions)))
print('This was achieved using {} search iterations.'.format(random_results.loc[0, 'iteration']))

from IPython.display import display
display(random_results.head(5))
## the best 5 parameters are displayed below

## end of random tuning module ##

{'class_weight': 'balanced', 'boosting_type': 'dart', 'num_leaves': 105, 'learning_rate': 0.19489663914444297, 'subsample_for_bin': 140000, 'min_child_samples': 480, 'reg_alpha': 0.02040816326530612, 'reg_lambda': 0.673469387755102, 'colsample_bytree': 0.7333333333333333, 'subsample': 0.98989898989899, 'metric': 'auc', 'verbose': 1}
The best model from random search scores 0.6738 on the test data.
This was achieved using 4 search iterations.


,loss,params,iteration,estimators,time
0,0.303164,"{'class_weight': 'balanced', 'boosting_type': ...",4,388,4.16774
1,0.304249,"{'class_weight': 'balanced', 'boosting_type': ...",6,242,0.84975
2,0.307988,"{'class_weight': None, 'boosting_type': 'dart'...",0,14,1.4
3,0.309646,"{'class_weight': 'balanced', 'boosting_type': ...",8,79,1.11357
4,0.311354,"{'class_weight': 'balanced', 'boosting_type': ...",1,47,0.473828


In [10]:
## Bayesian Hyperparameter Optimization using Hyperopt For LIGHTGBM ##

## start of random tuning module ##

## creating the objective function (to return the minimization value)
def objective(params, n_folds = N_FOLDS):
    """Objective function for Gradient Boosting Machine Hyperparameter Optimization"""
    
    # Keep track of evals
    global ITERATION
    
    ITERATION += 1
    
    # Retrieve the subsample if present otherwise set to 1.0
    subsample = params['boosting_type'].get('subsample', 1.0)
    
    # Extract the boosting type
    params['boosting_type'] = params['boosting_type']['boosting_type']
    params['subsample'] = subsample
    
    # Make sure parameters that need to be integers are integers
    for parameter_name in ['num_leaves', 'subsample_for_bin', 'min_child_samples']:
        params[parameter_name] = int(params[parameter_name])
    
    start = timer()
    
    # Perform n_folds cross validation
    cv_results = lgb.cv(params, train_set, num_boost_round = 10000, nfold = N_FOLDS, 
                        early_stopping_rounds = 100, metrics = 'auc', seed = random_seed)
    
    run_time = timer() - start
    
    # Extract the best score
    best_score = np.max(cv_results['auc-mean'])
    
    # Loss must be minimized
    loss = 1 - best_score
    
    # Boosting rounds that returned the highest cv score
    n_estimators = int(np.argmax(cv_results['auc-mean']) + 1)

    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a', newline='')
    writer = csv.writer(of_connection)
    writer.writerow([loss, params, ITERATION, n_estimators, run_time])
    
    # Dictionary with information for evaluation
    return {'loss': loss, 'params': params, 'iteration': ITERATION,
            'estimators': n_estimators, 
            'train_time': run_time, 'status': STATUS_OK}

## defining the domain space ##
# Define the search space (the hyper-parameter space to search for with respect to each parameter)
space = {
    'class_weight': hp.choice('class_weight', [None, 'balanced']),
    'boosting_type': hp.choice('boosting_type', [{'boosting_type': 'gbdt', 'subsample': hp.uniform('gdbt_subsample', 0.4, 1)}, 
                                                 {'boosting_type': 'dart', 'subsample': hp.uniform('dart_subsample', 0.4, 1)},
                                                 {'boosting_type': 'goss', 'subsample': 1.0}]),
    'num_leaves': hp.quniform('num_leaves', 30, 500, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.001), np.log(0.2)),
    'subsample_for_bin': hp.quniform('subsample_for_bin', 20000, 300000, 20000),
    'min_child_samples': hp.quniform('min_child_samples', 20, 100, 5),
    'reg_alpha': hp.uniform('reg_alpha', 0.0, 1.0),
    'reg_lambda': hp.uniform('reg_lambda', 0.0, 1.0),
    'colsample_bytree': hp.uniform('colsample_by_tree', 0.6, 0.9)
}

# optimization algorithm
tpe_algorithm = tpe.suggest

# Keep track of results
bayes_trials = Trials()

# File to save first results
out_file = 'gbm_trials.csv'
of_connection = open(out_file, 'w')
writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'iteration', 'estimators', 'train_time'])
of_connection.close()

In [11]:
%%capture

# Global variable
global  ITERATION

ITERATION = 0

# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials, rstate = np.random.RandomState(random_seed))

In [12]:
results = pd.read_csv('gbm_trials.csv')

# Sort with best scores on top and reset index for slicing
results.sort_values('loss', ascending = True, inplace = True)
results.reset_index(inplace = True, drop = True)
results.head()

,loss,params,iteration,estimators,train_time
0,0.301069,"{'boosting_type': 'gbdt', 'class_weight': 'bal...",7,338,2.334882
1,0.301278,"{'boosting_type': 'gbdt', 'class_weight': 'bal...",1,292,2.265558
2,0.303245,"{'boosting_type': 'gbdt', 'class_weight': 'bal...",10,210,2.032111
3,0.304062,"{'boosting_type': 'dart', 'class_weight': 'bal...",9,193,4.221925
4,0.304086,"{'boosting_type': 'gbdt', 'class_weight': 'bal...",4,45,1.056406


In [13]:
# Extract the ideal number of estimators and hyperparameters
best_bayes_estimators = int(results.loc[0, 'estimators'])
best_bayes_params = ast.literal_eval(results.loc[0, 'params']).copy()

# Re-create the best model and train on the training data
best_bayes_model = lgb.LGBMClassifier(n_estimators=best_bayes_estimators, n_jobs = -1, 
                                       objective = 'binary', random_state = 50, **best_bayes_params)
best_bayes_model.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
        colsample_bytree=0.6914076463501079,
        learning_rate=0.0010504530948152394, max_depth=-1, metric='auc',
        min_child_samples=70, min_child_weight=0.001, min_split_gain=0.0,
        n_estimators=338, n_jobs=-1, num_leaves=123, objective='binary',
        random_state=50, reg_alpha=0.5978444063317354,
        reg_lambda=0.43145877949439737, silent=True,
        subsample=0.7316723211407649, subsample_for_bin=300000,
        subsample_freq=0, verbose=1)

In [14]:
# Evaluate on the testing data 
preds = best_bayes_model.predict_proba(X_test)[:, 1]
print('The best model from Bayes optimization scores {:.5f} AUC ROC on the test set.'.format(roc_auc_score(y_test, preds)))
print('This was achieved after {} search iterations'.format(results.loc[0, 'iteration']))

The best model from Bayes optimization scores 0.67834 AUC ROC on the test set.
This was achieved after 7 search iterations


In [15]:
# Continue training
ITERATION = MAX_EVALS + 1

# Set more evaluations
MAX_EVALS = 100

In [16]:
%%capture

# Use the same trials object to keep training
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = MAX_EVALS, trials = bayes_trials, verbose = 1, rstate = np.random.RandomState(50))

In [17]:
results = pd.read_csv('gbm_trials.csv')

# Sort values with best on top and reset index for slicing
results.sort_values('loss', ascending = True, inplace = True)
results.reset_index(inplace = True, drop = True)

# Extract the ideal number of estimators and hyperparameters
best_bayes_estimators = int(results.loc[0, 'estimators'])
best_bayes_params = ast.literal_eval(results.loc[0, 'params']).copy()

# Re-create the best model and train on the training data
best_bayes_model = lgb.LGBMClassifier(n_estimators=best_bayes_estimators, n_jobs = -1, 
                                       objective = 'binary', random_state = 50, **best_bayes_params)
best_bayes_model.fit(X_train, y_train)

# Evaluate on the testing data 
preds = best_bayes_model.predict_proba(X_test)[:, 1]
print('The best model from Bayes optimization scores {:.5f} AUC ROC on the test set.'.format(roc_auc_score(y_test, preds)))
print('This was achieved after {} search iterations'.format(results.loc[0, 'iteration']))

from IPython.display import display
display(random_results.head(5))
## the best 5 parameters are displayed below

The best model from Bayes optimization scores 0.68398 AUC ROC on the test set.
This was achieved after 80 search iterations


,loss,params,iteration,estimators,time
0,0.303164,"{'class_weight': 'balanced', 'boosting_type': ...",4,388,4.16774
1,0.304249,"{'class_weight': 'balanced', 'boosting_type': ...",6,242,0.84975
2,0.307988,"{'class_weight': None, 'boosting_type': 'dart'...",0,14,1.4
3,0.309646,"{'class_weight': 'balanced', 'boosting_type': ...",8,79,1.11357
4,0.311354,"{'class_weight': 'balanced', 'boosting_type': ...",1,47,0.473828
